Projeto da Diciplina PROGRAMAÇÃO ESTATÍSTICA COM PYTHON   
Grupo:     
MATHEUS ANTONIO ALVES DE ARAUJO     
NATANAEL FERNANDES SARMENTO     


Os dados obtidos de Covid-19 no brasil apresenta ultima atualização na data de 30/06/2021,  esse dados foram obtidos no site url:https://brasil.io/dataset/covid19/caso_full/. Dos dados obtidos foi feito um tratamento no banco e uma filtragem por estado e região assim foi salvo um dicionario de cada estado e região apos todo o tratamento aplicado no banco, a segir vemos uma descrição das variaveis dos bancos salvos.
+ state: sigla da unidade federativa, exemplo: SP.
+ new_confirmed: número de novos casos confirmados .
+ new_deaths: número de novos óbitos.
+ date: data de coleta dos dados no formato YYYY-MM-DD.
+ order_for_place: 	Dias a partir do 1º caso de covid registrado para este local. 
+ last_available_confirmed:	Número de casos confirmados acumulados.
+ last_available_deaths: Óbitos acumulados

Para verificar as demais variaveis apresentadas no banco completo olhar a descrição no site.


A seguir vamos tratar o banco de dados agrupando por estado e região, assim calculando a incidência, mortalidade, IAD.

In [159]:
import numpy as np
import pandas as pd

url = '/content/drive/MyDrive/Colab Notebooks/PROGRAMAÇÃO ESTATÍSTICA COM PYTHON/Projeto/covid19_21_07.csv'
covid = pd.read_csv(url, index_col=1, parse_dates=True, sep=',')
covid["month"]=covid.index.strftime('%Y-%m')

state=set(covid['state'])
desc_pop=pd.DataFrame(index=state,columns=['order_for_place','estimated_population'])
for i in state:
  desc_pop.loc[i]=covid[covid['state']==i].iloc[0,[1,12]]
  a=covid[covid['state']==i].groupby(by=["month"]).sum()
  b=covid[covid['state']==i].groupby(by=["month"]).max()
  Dados[i]=pd.concat([b.loc[b.index,['last_available_confirmed','last_available_deaths']],
                     a.loc[a.index,['new_confirmed','new_deaths']]], axis=1)

regiao={'Sul':['PR','RS','SC'],'Centro-Oeste':['DF','MT','GO','MS'],
        'Norte':['AP','RR','PA','RO','AC','AM','TO'],
        'Nordeste':['PE','AL','SE','PI','CE','RN','MA','BA','PB'],
        'Sudeste':['SP','ES','RJ','MG']}
for j in regiao:
  desc_pop.loc[j]=[desc_pop.loc[regiao[j],'order_for_place'].max(),
                   desc_pop.loc[regiao[j],'estimated_population'].sum()]
  dados=covid[covid['state'].isin(regiao[j])]
  dados=dados.groupby(dados.index).sum()
  dados["month"]=dados.index.strftime('%Y-%m')
  a=dados.groupby(by=["month"]).sum()
  b=dados.groupby(by=["month"]).max()
  Dados[j]=pd.concat(
      [b.loc[b.index,['last_available_confirmed','last_available_deaths']],
      a.loc[a.index,['new_confirmed','new_deaths']]], axis=1)

dados=covid.groupby(covid.index).sum()
dados["month"]=dados.index.strftime('%Y-%m')
a=dados.groupby(by=["month"]).sum()
b=dados.groupby(by=["month"]).max()
Dados['Brasil']=pd.concat(
    [b.loc[b.index,['last_available_confirmed','last_available_deaths']],
    a.loc[a.index,['new_confirmed','new_deaths']]], axis=1)

desc_pop.loc['Brasil']=[desc_pop.loc[state,'order_for_place'].max(),
                        desc_pop.loc[state,'estimated_population'].sum()]

for cada in Dados:
  desc_pop.loc[cada,'Incidencia']=(Dados[cada].loc['2021-07','last_available_confirmed']/desc_pop.loc[cada,'estimated_population'])*100000
  desc_pop.loc[cada,'Mortalidade']=(Dados[cada].loc['2021-07','last_available_deaths']/Dados[cada].loc['2021-07','last_available_confirmed'])*100000
  desc_pop.loc[cada,'IAD']=(Dados[cada].loc['2021-07','last_available_confirmed']/desc_pop.loc[cada,'order_for_place'])
  desc_pop.loc[cada,'Casos_Confirmados']=Dados[cada].loc['2021-07','last_available_confirmed']
  desc_pop.loc[cada,'óbtos']=Dados[cada].loc['2021-07','last_available_deaths']
#[1,5,6,2,3,4,0]
desc_pop=desc_pop.loc[desc_pop.index,['estimated_population','Casos_Confirmados','óbtos','Incidencia','Mortalidade','IAD','order_for_place']]
desc_pop

,estimated_population,Casos_Confirmados,óbtos,Incidencia,Mortalidade,IAD,order_for_place
CE,9187103,894678.0,22791.0,9738.412642,2547.396941,1871.711297,478
RN,3534165,347248.0,6853.0,9825.460894,1973.517486,720.431535,482
BA,14930634,1141612.0,24428.0,7646.105316,2139.781292,2339.368852,488
MS,2809394,339323.0,8400.0,12078.156357,2475.517427,706.922917,480
MG,21292666,1836198.0,47148.0,8623.617165,2567.696948,3778.185185,486
TO,1590248,200243.0,3266.0,12591.935346,1631.018313,420.678571,476
AP,861773,118066.0,1857.0,13700.359607,1572.849084,249.084388,474
SE,2318822,266590.0,5773.0,11496.785868,2165.497581,555.395833,480
PA,8690745,558861.0,15646.0,6430.530409,2799.622804,1174.077731,476
SC,7252502,1066484.0,17146.0,14705.049375,1607.712821,2212.622407,482


A seguir vamos fazer mais coisas